In [ ]:
import json
import pandas as pd
from collections import Counter
from collections import defaultdict
import re

In [ ]:
docs = []
ht_occurences = Counter([])
ht_texts = defaultdict(list)
lang_occurences = Counter([])

j=0
with open(r"C:\Users\Shmuli\Downloads\twt_relevant_13_05_20.json", encoding='utf-8') as f:
    try:
        for i, line in enumerate(f):
           # print('line:' ,i ," : ", line)
            line = line.replace(",\n","")
            try:
                data = json.loads(line)
                full_text = data['doc']['full_text'].lower()
                if 'coronavirus' in full_text or 'covid' in full_text or True:
                    docs.append(data['doc'])
                    if 'relevance' in data['doc']:
                        lang_occurences[data['doc']['lang']] += 1
                        ht = re.findall(r"#(\w+)",data['doc']['full_text'].lower())
                        for hash_tag in ht:
                            ht_occurences[hash_tag] += 1
                            ht_texts[hash_tag].append(j)
                    j+=1     
            except ValueError as e:
                print(f"Malformed JSON at {i}")
              
                
    except Exception as e:
        print("Exception: ", e)

In [ ]:
len(docs)

In [ ]:
ht_occurences.most_common(50)

In [ ]:
lang_occurences.most_common(20)

In [ ]:
-(df.coordinates.isnull().sum()-df.shape[0])/df.shape[0]

In [ ]:
df.keywords.value_counts()

In [ ]:
place_type_all = Counter()
point_place_mask = []
point_mask = []
point_user_mask = []
for i,row in df.iterrows():
    pt = row['coordinates']['type']
    place_type_all[pt]+=1
    if pt =="Point_Place":
        point_place_mask.append(i)
    elif pt == "Point":
        point_mask.append(i)
    elif pt=="Point_User":
        point_user_mask.append(i)
    else:
        print('error')
    

In [ ]:
place_type_all

In [ ]:
df.sort_values('created_at')

In [ ]:
user_counts = Counter()
for i,row in df.iterrows():
    user_id = row['user']['id']
    user_name = row['user']['name']
    user_counts[user_id]+=1
    

In [ ]:
user_counts.most_common(100)

In [ ]:
yesterday = pd.to_datetime('01 Feb 2020', format = '%d %b %Y')

In [ ]:
df[df['created_at']>=yesterday]

In [ ]:
location_counts = Counter()
for i,row in df.iterrows():
    location = tuple(row['coordinates']['coordinates'])
    location_counts[location] +=1
    


In [ ]:
location_counts.most_common(10)

In [ ]:
df['location_type'] = df.apply(lambda x: x['coordinates']['type'], axis=1)

In [ ]:
df['location_type']

In [ ]:
df_cities = df[df['location_type']== "Point_User"].copy()

In [ ]:
df_cities

In [ ]:
[((-33.8688, 151.2093), 29042),
 ((-37.810151, 144.965594), 26220),
 ((-27.4698, 153.0251), 20256),
 ((-31.9505, 115.8605), 16001),

In [ ]:
def get_city(city_coordinates):
    
    if city_coordinates == [-33.8688, 151.2093]:
        return 'Sydney'
    elif city_coordinates == [-37.810151, 144.965594]:
        return 'Melbourne'
    
    elif city_coordinates == [-27.4698, 153.0251]:
        return 'Brisbane'

    elif city_coordinates == [-31.9505, 115.8605]:
        return 'Perth'
    
    

In [ ]:
df_cities['City'] = df_cities.apply(lambda x: get_city(x['coordinates']['coordinates']), axis=1)

In [ ]:
df_cities

In [ ]:

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from collections import Counter  
from nltk.stem import WordNetLemmatizer 
import string

def get_ngrams(df, n):
    
    lemmatizer = WordNetLemmatizer() 
    stop_words = set(stopwords.words('english')) 
    stop_words.add("https")

    total_counts = Counter()
    tweet_counts = Counter()
    
    raw_tweet_texts = df["full_text"].values
    
    for i, tweet in enumerate(raw_tweet_texts):
        
        word_tokens = word_tokenize(tweet.lower()) 
        
        filtered_sentence = [lemmatizer.lemmatize(w) for w in word_tokens if not w in stop_words and w not in string.punctuation] 
        filtered_sentence_set = set(filtered_sentence)
    
        total_counts.update(filtered_sentence)
        tweet_counts.update(filtered_sentence_set)
        
    return tweet_counts
    
        

In [ ]:
df_melb = df_cities[df_cities["City"]=='Melbourne'].copy()

In [ ]:
df_melb.shape

In [ ]:
tweet_counts_melb = get_ngrams(df_melb,1)

In [ ]:
tweet_counts_melb.most_common(100)

In [ ]:
cities = ["Melbourne","Sydney", "Brisbane", "Perth"]
cities_tweet_counts = dict()
df_ = dict()
for city in cities:
    df_[city] = df_cities[df_cities["City"]==city].copy()
    cities_tweet_counts[city] = get_ngrams(df_[city],1)

In [ ]:
df_["Melbourne"].shape

In [ ]:
cities_tweet_counts["Melbourne"].most_common(100)

In [ ]:
df_["Sydney"].shape

In [ ]:
cities_tweet_counts["Sydney"].most_common(100)

In [ ]:
cities_tweet_counts["Brisbane"].most_common(100)

In [ ]:
cities_tweet_counts["Perth"].most_common(100)

In [ ]:
def get_common_topics(n):
    
    common_topics = set([x for x,y in cities_tweet_counts["Melbourne"].most_common(n)])
    for city in cities:
        common_topics = common_topics.intersection(set([x for x,y in cities_tweet_counts[city].most_common(n)]))
        
    return common_topics

In [ ]:
get_common_topics(200) ['app','auspol','business','care','case','china','chinese','community','confirmed', 'covidsafe','death','distancing','economy','economic','hospital','infection','job','lockdown','minister','morrison','outbreak','pandemic','positive','response','restriction','safe','school','scottmorissonmp','vaccine','virus','work','worker','working']

In [ ]:

keywords = ['app','auspol','business','care','case','china','chinese','community','confirmed'
                        , 'covidsafe','death','distancing','economy','economic','hospital','infection','job','lockdown',
                        'minister','morrison','outbreak','pandemic','positive','response','restriction','safe','school',
                        'scottmorissonmp','vaccine','virus','work','worker','working']
df_rows= []
for keyword in keywords:
    for city in cities:
        count = cities_tweet_counts[city][keyword]/df_[city].shape[0]*100
        row = [keyword, count, city]
        df_rows.append(row)
        
        
keyword_df = pd.DataFrame(df_rows, columns = ["keyword", "count", "city"] )
    
    

In [ ]:
keyword_df

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(30,30))
plt.xticks(rotation=90)
sns.barplot(x="keyword", y="count", hue="city", data=keyword_df, ax=ax)

In [ ]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from collections import Counter  
from nltk.stem import WordNetLemmatizer 
import string
#string.punctuation
  
lemmatizer = WordNetLemmatizer() 

stop_words = set(stopwords.words('english')) 
stop_words.add("https")

total_counts = Counter()
tweet_counts = Counter()

raw_tweet_texts = df["full_text"].values
cv_tokens =  Counter()
for i, tweet in enumerate(raw_tweet_texts):
    
    word_tokens = word_tokenize(tweet.lower()) 
    if 'coronavirus' in word_tokens:
       # print(word_tokens)
        #print(tweet)
        word_tokens2 = tweet.lower().split(" ")
        for token in word_tokens2:
            if 'coronavirus' in token:
                cv_tokens[token] +=1
        #print("\n\n")
       
    filtered_sentence = [lemmatizer.lemmatize(w) for w in word_tokens if not w in stop_words and w not in string.punctuation] 
    filtered_sentence_set = set(filtered_sentence)
    
    total_counts.update(filtered_sentence)
    tweet_counts.update(filtered_sentence_set)
    
    if i % 50000 == 0:
        print('tweet: ', i)
    
    

In [ ]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from collections import Counter  
from nltk.stem import WordNetLemmatizer 
import string
#string.punctuation
  
lemmatizer = WordNetLemmatizer() 

stop_words = set(stopwords.words('english')) 
stop_words.add("https")

bi_total_counts = Counter()
bi_tweet_counts = Counter()

raw_tweet_texts = df["full_text"].values

for i, tweet in enumerate(raw_tweet_texts):
    
    word_tokens = word_tokenize(tweet.lower()) 
    #word_tokens = tweet.split(" ")
    #print(word_tokens)
    sent = " ".join([lemmatizer.lemmatize(w) for w in word_tokens if not w in stop_words and w not in string.punctuation])
    #print(sent)
    bigram_tokens = generate_ngrams(sent,2)
    bigram_tokens_set = set(bigram_tokens)
    
    bi_total_counts.update(bigram_tokens)
    bi_tweet_counts.update(bigram_tokens_set)
    
    if i % 50000 == 0:
        print('tweet: ', i)
    
    